In [1]:
import os

In [2]:
%pwd

'/Users/divyanshu9871gmail.com/Desktop/ml-ops-holiday-package-prediction/notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/divyanshu9871gmail.com/Desktop/ml-ops-holiday-package-prediction'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = (
    "https://dagshub.com/Divyanshu9822/ml-ops-holiday-package-prediction.mlflow"
)

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    preprocessor_model_path: Path
    model_path: Path
    model_name: str
    model_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.constants import *
from src.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        model_config_filepath=MODEL_CONFIG_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):

        self.config = read_yaml(config_filepath)
        self.model_config = read_yaml(model_config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        model_name = self.model_config.model
        params = self.model_config.params
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            preprocessor_model_path=config.preprocessor_model_path,
            model_path=config.model_path,
            model_name=model_name,
            model_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Divyanshu9822/ml-ops-holiday-package-prediction.mlflow",
        )

In [9]:
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import xgboost

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]=""
os.environ["MLFLOW_TRACKING_USERNAME"]=""
os.environ["MLFLOW_TRACKING_PASSWORD"]=""

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, y_test, y_test_pred):
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, average="weighted")
        precision = precision_score(y_test, y_test_pred, average="weighted")
        recall = recall_score(y_test, y_test_pred, average="weighted")
        roc_auc = roc_auc_score(y_test, y_test_pred)
        return accuracy, f1, precision, recall, roc_auc

    def log_into_mlflow(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        preprosessor = joblib.load(self.config.preprocessor_model_path)
        test_x_transformed = preprosessor.transform(test_x)

        # Set MLflow URI and start a new run
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Make predictions
            pred_y = model.predict(test_x_transformed)

            # Evaluate metrics
            accuracy, f1, precision, recall, roc_auc = self.eval_metrics(test_y, pred_y)

            # Save metrics to JSON
            scores = {
                "accuracy": accuracy,
                "f1_score": f1,
                "precision": precision,
                "recall": recall,
                "roc_auc": roc_auc,
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics to MLflow
            mlflow.log_params(self.config.model_params)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("roc_auc", roc_auc)

            # Log the model
            if tracking_url_type_store != "file":
                if isinstance(model, xgboost.XGBModel):
                    mlflow.xgboost.log_model(
                        model, "model", registered_model_name=self.config.model_name
                    )
                else:
                    mlflow.sklearn.log_model(
                        model, "model", registered_model_name=self.config.model_name
                    )
            else:
                if isinstance(model, xgboost.XGBModel):
                    mlflow.xgboost.log_model(model, "model")
                else:
                    mlflow.sklearn.log_model(model, "model")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-09-09 14:03:49,816: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-09 14:03:49,820: INFO: common: yaml file: model_config.yaml loaded successfully]
[2024-09-09 14:03:49,823: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-09 14:03:49,824: INFO: common: created directory at: artifacts]
[2024-09-09 14:03:49,825: INFO: common: created directory at: artifacts/model_evaluation]
[2024-09-09 14:03:50,864: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/Users/divyanshu9871gmail.com/Desktop/ml-ops-holiday-package-prediction/venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [14:03:53] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
/Users/divyanshu9871gmail.com/Desktop/ml-ops-holiday-package-prediction/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'Xgboost' already exists. Creating a new version of this model...
2024/09/09 14:04:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finis